# 제품 이상여부 판별 프로젝트


### 데이터 읽어오기


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

/home/elicer/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

RANDOM_STATE = 110

train_data = pd.read_csv("train_data_0827.csv")
test_data = pd.read_csv("test_data_0827.csv")

In [4]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

In [5]:
# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [6]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [7]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [8]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [9]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [10]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 38
train_data_dam DataFrame의 칼럼 수: 21
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 13
train_data_fill2 DataFrame의 칼럼 수: 13
----test data-----
test_data DataFrame의 칼럼 수: 39
test_data_dam DataFrame의 칼럼 수: 22
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 14
test_data_fill2 DataFrame의 칼럼 수: 14


---

## Optuna

스레스홀드 0.3으로 맞춘상태에서 튜닝 진행한 것

In [11]:
RANDOM_STATE = 110
THRESHOLD = 0.3

In [12]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    
    # 'Normal'과 'AbNormal'을 숫자로 변환
    y_tr = y_tr.map({'Normal': 0, 'AbNormal': 1})
    y_val = y_val.map({'Normal': 0, 'AbNormal': 1})
    
    param = {
        'iterations': trial.suggest_int('iterations', 800, 5000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.2),
        'depth': trial.suggest_int('depth', 4, 13),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10, log=True),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.1, 10),
        'border_count': trial.suggest_int('border_count', 128, 300),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        
        'random_seed': RANDOM_STATE,
        'eval_metric': 'F1',
        'logging_level': 'Silent',
        'boosting_type': 'Plain'
    }

    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    pred = (pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측
    
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    train_data_autoclave.drop("target", axis=1),  
    train_data_autoclave["target"],               
    test_size=0.2,
    shuffle=True,
    random_state=RANDOM_STATE,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=400)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))

[I 2024-08-26 05:10:53,577] A new study created in memory with name: no-name-43086764-76a7-4611-9bbd-32a0a954d18f
[I 2024-08-26 05:10:58,648] Trial 0 finished with value: 0.1727574750830565 and parameters: {'iterations': 848, 'learning_rate': 0.13339879944049993, 'depth': 7, 'l2_leaf_reg': 1.099256962998379, 'random_strength': 8.185937574992959, 'bagging_temperature': 6.85638497930364, 'border_count': 338, 'scale_pos_weight': 6.735006454525373, 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.1727574750830565.
[I 2024-08-26 05:11:05,153] Trial 1 finished with value: 0.19374999999999998 and parameters: {'iterations': 1371, 'learning_rate': 0.031403208613589216, 'depth': 3, 'l2_leaf_reg': 0.5027927228085797, 'random_strength': 5.961831701274088, 'bagging_temperature': 0.31183600257184896, 'border_count': 227, 'scale_pos_weight': 1.698550183782988, 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.19374999999999998.
[I 2024-08-26 05:11:16,580] Trial 2 finished with 

[I 2024-08-26 05:20:38,532] Trial 20 finished with value: 0.17777777777777778 and parameters: {'iterations': 3144, 'learning_rate': 0.10689758387605522, 'depth': 7, 'l2_leaf_reg': 2.5393556872946537, 'random_strength': 7.167675058692504, 'bagging_temperature': 3.396153219420854, 'border_count': 159, 'scale_pos_weight': 6.579644269585977, 'grow_policy': 'Depthwise'}. Best is trial 13 with value: 0.21663019693654267.
[I 2024-08-26 05:20:57,595] Trial 21 finished with value: 0.21534653465346532 and parameters: {'iterations': 3231, 'learning_rate': 0.024185656262513212, 'depth': 4, 'l2_leaf_reg': 4.285254072181592, 'random_strength': 2.3190311316595964, 'bagging_temperature': 4.897522886241988, 'border_count': 337, 'scale_pos_weight': 1.9121774554278088, 'grow_policy': 'Lossguide'}. Best is trial 13 with value: 0.21663019693654267.
[I 2024-08-26 05:21:17,862] Trial 22 finished with value: 0.22741764080765145 and parameters: {'iterations': 2728, 'learning_rate': 0.0200321563125834, 'depth':

[I 2024-08-26 05:25:36,452] Trial 40 finished with value: 0.2073778664007976 and parameters: {'iterations': 2459, 'learning_rate': 0.05089064938367791, 'depth': 10, 'l2_leaf_reg': 1.3241383866527572, 'random_strength': 1.2147969015142048, 'bagging_temperature': 9.97160170412267, 'border_count': 230, 'scale_pos_weight': 1.4694772037569643, 'grow_policy': 'SymmetricTree'}. Best is trial 22 with value: 0.22741764080765145.
[I 2024-08-26 05:25:46,434] Trial 41 finished with value: 0.22788203753351205 and parameters: {'iterations': 2362, 'learning_rate': 0.03942998728788136, 'depth': 6, 'l2_leaf_reg': 2.0609550730489197, 'random_strength': 1.0212166453035267, 'bagging_temperature': 8.628613119338489, 'border_count': 248, 'scale_pos_weight': 1.610261377895462, 'grow_policy': 'SymmetricTree'}. Best is trial 41 with value: 0.22788203753351205.
[I 2024-08-26 05:25:57,783] Trial 42 finished with value: 0.22474460839954596 and parameters: {'iterations': 2321, 'learning_rate': 0.04674278545053326,

[I 2024-08-26 05:34:54,173] Trial 60 finished with value: 0.15718717683557396 and parameters: {'iterations': 2228, 'learning_rate': 0.1083311165614983, 'depth': 5, 'l2_leaf_reg': 0.10014096835705137, 'random_strength': 1.216134227696803, 'bagging_temperature': 9.032261964791346, 'border_count': 277, 'scale_pos_weight': 7.385926311862386, 'grow_policy': 'SymmetricTree'}. Best is trial 52 with value: 0.23390275952693823.
[I 2024-08-26 05:35:06,264] Trial 61 finished with value: 0.21870882740447958 and parameters: {'iterations': 2453, 'learning_rate': 0.042526906851625725, 'depth': 7, 'l2_leaf_reg': 1.6336518889876046, 'random_strength': 0.7851946226285821, 'bagging_temperature': 9.722088649214975, 'border_count': 243, 'scale_pos_weight': 1.3322098781349396, 'grow_policy': 'SymmetricTree'}. Best is trial 52 with value: 0.23390275952693823.
[I 2024-08-26 05:35:19,348] Trial 62 finished with value: 0.21666666666666665 and parameters: {'iterations': 2631, 'learning_rate': 0.06179090920309425

[I 2024-08-26 05:39:24,712] Trial 80 finished with value: 0.09663865546218486 and parameters: {'iterations': 943, 'learning_rate': 0.010675866848991238, 'depth': 4, 'l2_leaf_reg': 0.9005762554558093, 'random_strength': 11.246693542462172, 'bagging_temperature': 7.15913234374724, 'border_count': 310, 'scale_pos_weight': 1.0259320878895972, 'grow_policy': 'Lossguide'}. Best is trial 72 with value: 0.2362204724409449.
[I 2024-08-26 05:39:44,111] Trial 81 finished with value: 0.22878228782287824 and parameters: {'iterations': 2590, 'learning_rate': 0.023990245371992618, 'depth': 6, 'l2_leaf_reg': 0.8525652114078384, 'random_strength': 11.996675823341407, 'bagging_temperature': 9.660994810620235, 'border_count': 318, 'scale_pos_weight': 1.550839778364074, 'grow_policy': 'Lossguide'}. Best is trial 72 with value: 0.2362204724409449.
[I 2024-08-26 05:40:05,689] Trial 82 finished with value: 0.22155688622754488 and parameters: {'iterations': 2894, 'learning_rate': 0.023579123982935647, 'depth'

...

[I 2024-08-26 06:34:31,207] Trial 320 finished with value: 0.22031823745410037 and parameters: {'iterations': 2629, 'learning_rate': 0.06073908863671, 'depth': 7, 'l2_leaf_reg': 3.05254868701933, 'random_strength': 7.62532767403173, 'bagging_temperature': 6.9118340068013, 'border_count': 203, 'scale_pos_weight': 1.4171170022332582, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:34:39,173] Trial 321 finished with value: 0.23827629911280102 and parameters: {'iterations': 1642, 'learning_rate': 0.0487395931540137, 'depth': 7, 'l2_leaf_reg': 1.8185079373310082, 'random_strength': 7.283258177314755, 'bagging_temperature': 7.240907884051836, 'border_count': 183, 'scale_pos_weight': 1.6007768671541855, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:34:48,421] Trial 322 finished with value: 0.2234392113910186 and parameters: {'iterations': 1561, 'learning_rate': 0.04801813886270423, 'dep

[I 2024-08-26 06:38:22,358] Trial 340 finished with value: 0.22446916076845297 and parameters: {'iterations': 1598, 'learning_rate': 0.043305569474931346, 'depth': 7, 'l2_leaf_reg': 2.9337349268777966, 'random_strength': 7.459862489164186, 'bagging_temperature': 5.933072291699232, 'border_count': 77, 'scale_pos_weight': 2.299003008211068, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:38:29,875] Trial 341 finished with value: 0.22195704057279236 and parameters: {'iterations': 1770, 'learning_rate': 0.05374899456510738, 'depth': 6, 'l2_leaf_reg': 2.594968094346286, 'random_strength': 5.866517180427904, 'bagging_temperature': 5.467489533445012, 'border_count': 219, 'scale_pos_weight': 1.9560190249943825, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:38:42,042] Trial 342 finished with value: 0.23129251700680273 and parameters: {'iterations': 2502, 'learning_rate': 0.046537675622511

[I 2024-08-26 06:41:27,218] Trial 360 finished with value: 0.22290076335877865 and parameters: {'iterations': 1723, 'learning_rate': 0.045142583786526164, 'depth': 7, 'l2_leaf_reg': 2.6451164852366538, 'random_strength': 7.7578952776189505, 'bagging_temperature': 7.4678395278034335, 'border_count': 182, 'scale_pos_weight': 1.1620616223709055, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:41:35,098] Trial 361 finished with value: 0.22556390977443608 and parameters: {'iterations': 1618, 'learning_rate': 0.05080633615955797, 'depth': 7, 'l2_leaf_reg': 1.8977534322570457, 'random_strength': 6.9087769431649, 'bagging_temperature': 7.153089260737095, 'border_count': 219, 'scale_pos_weight': 1.668346887449059, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:41:43,686] Trial 362 finished with value: 0.22646657571623466 and parameters: {'iterations': 1785, 'learning_rate': 0.0394644974625

[I 2024-08-26 06:46:42,111] Trial 380 finished with value: 0.22849807445442874 and parameters: {'iterations': 1769, 'learning_rate': 0.056100416770391556, 'depth': 7, 'l2_leaf_reg': 2.536458692899726, 'random_strength': 7.7457769458757975, 'bagging_temperature': 7.767460692332395, 'border_count': 217, 'scale_pos_weight': 1.491946619506014, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:46:49,886] Trial 381 finished with value: 0.22754491017964074 and parameters: {'iterations': 1615, 'learning_rate': 0.04533813384378449, 'depth': 7, 'l2_leaf_reg': 2.690204125862691, 'random_strength': 8.301740256982438, 'bagging_temperature': 7.545050075241579, 'border_count': 189, 'scale_pos_weight': 1.9402911028047118, 'grow_policy': 'SymmetricTree'}. Best is trial 191 with value: 0.24528301886792453.
[I 2024-08-26 06:46:57,501] Trial 382 finished with value: 0.2148997134670487 and parameters: {'iterations': 1828, 'learning_rate': 0.026436743533909

Best trial: score 0.24528301886792453, 
params {'iterations': 2453, 'learning_rate': 0.045101812655324314, 'depth': 7, 'l2_leaf_reg': 2.7766650741754, 'random_strength': 7.514157578279006, 'bagging_temperature': 7.736554866068651, 'border_count': 301, 'scale_pos_weight': 1.73893896002959, 'grow_policy': 'SymmetricTree'}


1. 0.24528301886792453

params {'iterations': 2453, 'learning_rate': 0.045101812655324314, 'depth': 7, 'l2_leaf_reg': 2.7766650741754, 'random_strength': 7.514157578279006, 'bagging_temperature': 7.736554866068651, 'border_count': 301, 'scale_pos_weight': 1.73893896002959, 'grow_policy': 'SymmetricTree'}

In [13]:
# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name]

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

In [14]:
train_and_evaluate_model(
    'cat', train_data_autoclave,
    iterations = 2786, 
    learning_rate = 0.016342560305036093, 
    depth = 8, 
    min_data_in_leaf = ,
    l2_leaf_reg = 3.7187150890684246, 
    random_strength = 0.13164684607188099, 
    bagging_temperature = 9.823498597792092, 
    border_count = 158, 
    scale_pos_weight = 1.8735070170496537,
    grow_policy = 'SymmetricTree',

    random_state = RANDOM_STATE,
    eval_metric = 'F1',
    logging_level = 'Silent',
    boosting_type = 'Plain'
)

cat 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.24691358024691357
---
Confusion Matrix:
[[7392  270]
 [ 340  100]]
---
Accuracy: 0.9247099481609479
Precision: 0.2702702702702703
Recall: 0.22727272727272727


